In [14]:
import numpy as np
import cv2
from cv2 import dnn

In [ ]:
#-Model file paths
proto_file = 'Model\colorization_deploy_v2.prototxt'
model_file = 'Model\colorization_release_v2.caffemodel'
hull_pts = 'Model\pts_in_hull.npy'
img_path = 'images/img1.jpg'

In [ ]:
#-Reading the model params
net=dnn.readNetFromCaffe(proto_file,model_file)
kernel=np.load(hull_pts)

In [ ]:
#Reading and preprocessing image
img=cv2.imread(img_path)
scaled=img.astype("float32")/255.0
lab_img=cv2.cvtColor(scaled,cv2.COLOR_BGR2LAB)


In [ ]:

#clustering centers as 1x1 convolutions to the model
class8=net.getLayerId("class8_ab")
conv8=net.getLayerId("conv8_313_rh")
pts=kernel.transpose().reshape(2,313,1,1)
net.getLayer(class8).blobs=[pts.astype("float32")]
net.getLayer(conv8).blobs=[np.full([1,313],2.606,dtype="float32")]

In [ ]:
#resizing the image for the network
resized = cv2.resize(lab_img, (224, 224))
#L channel split 
L = cv2.split(resized)[0]
L -= 50 

In [ ]:
# predicting the ab channels from the input L channel
net.setInput(cv2.dnn.blobFromImage(L))
ab_channel = net.forward()[0, :, :, :].transpose((1, 2, 0))

In [ ]:
#resizing the predicted 'ab' volume to the same dimensions
ab_channel=cv2.resize(ab_channel,(img.shape[1],img.shape[0]))

In [ ]:
# Takeing the L channel from the image
L=cv2.split(lab_img)[0]
color=np.concatenate((L[:,:,np.newaxis],ab_channel),axis=2)

In [ ]:
#convert the image from Lab to BGR 
color=cv2.cvtColor(color, cv2.COLOR_LAB2BGR)
color=np.clip(color, 0, 1)

In [ ]:
#change the image to 0-255 range and convert it from float32 to int
color=(255*color).astype("uint8")

In [25]:
img=cv2.resize(img,(640,640))
color=cv2.resize(color,(640,640))

In [26]:
res=cv2.hconcat([img,color])
cv2.imshow("Graysacle->color",res)
cv2.waitKey(0)

-1